In [1]:
import csv
import pandas as pd

In [2]:
dfA = pd.read_csv("FB - A - arrumado.csv")
dfB = pd.read_csv("FB - B - arrumado.csv")
print(f"Tamanho dos datasets: {dfA.shape, dfB.shape}")

Tamanho dos datasets: ((63, 28), (63, 28))


In [3]:
dfA.head()

,id redação,id prompt,redação,Sintaxe,Desvios,nota,Tema,Tipologia,Conclusão,Repertório,...,Elementos coesivos,Repetições,Inadequações,Monobloco?,nota.3,Direitos humanos,Aderência,Elementos,Condicional,nota.4
0,103.html,a-cultura-do-cancelamento-nas-redes-sociais,"['Infelizmente muitos usuários anônimos, julga...",Sintaxe regular,Muitos desvios,2,Abordagem completa,Nenhuma parte embrionária,Conclusão é finalizada,Não legitimado,...,Presença regular de elementos coesivos inter e...,Raras ou ausentes repetições,Sem inadequações,Não é um monobloco,3,Proposta respeita os direitos humanos,Ausência de proposta ou cópia integral de prop...,Apenas elementos nulos,Não possui estrutura condicional,0
1,113.html,a-cultura-do-cancelamento-nas-redes-sociais,['O ser humano como ser pensante e racional é ...,Sintaxe regular,Poucos desvios,3,Abordagem completa,Nenhuma parte embrionária,Conclusão é finalizada,Legitimado,...,Presença regular de elementos coesivos inter e...,Raras ou ausentes repetições,Poucas inadequações,Não é um monobloco,3,Proposta respeita os direitos humanos,Proposta aborda completamente o tema,3 elementos válidos,Não possui estrutura condicional,3
2,33.html,a-cultura-do-cancelamento-nas-redes-sociais,['A cultura do cancelamento nas redes sociais ...,Sintaxe excelente,Poucos desvios,4,Abordagem completa,Nenhuma parte embrionária,Conclusão é finalizada,Legitimado,...,Presença constante de elementos coesivos inter...,Raras ou ausentes repetições,Poucas inadequações,Não é um monobloco,4,Proposta respeita os direitos humanos,Proposta aborda completamente o tema,3 elementos válidos,Não possui estrutura condicional,3
3,39.html,a-cultura-do-cancelamento-nas-redes-sociais,['A internet no dias de hoje ganhou muita infl...,Sintaxe regular,Poucos desvios,3,Abordagem completa,Nenhuma parte embrionária,Conclusão é finalizada,Legitimado,...,Presença regular de elementos coesivos inter e...,Poucas repetições,Sem inadequações,Não é um monobloco,3,Proposta respeita os direitos humanos,Proposta aborda completamente o tema,3 elementos válidos,Não possui estrutura condicional,3
4,41.html,a-cultura-do-cancelamento-nas-redes-sociais,['São Tomás de Aquino defendeu que todas as pe...,Sintaxe excelente,Poucos desvios,4,Abordagem completa,Nenhuma parte embrionária,Conclusão é finalizada,Legitimado,...,Presença constante de elementos coesivos inter...,Raras ou ausentes repetições,Sem inadequações,Não é um monobloco,4,Proposta respeita os direitos humanos,Proposta aborda completamente o tema,4 elementos válidos,Não possui estrutura condicional,4


In [4]:
print(f"Número de prompts: {len(dfA['id prompt'].unique())}")

Número de prompts: 7


In [5]:
prompts = dfA['id prompt'].unique().tolist()
count_prompts = [0]*len(prompts)
for i, row in dfA.iterrows():
    index = prompts.index(row['id prompt'])
    count_prompts[index] += 1
print(count_prompts)

[9, 9, 9, 9, 9, 9, 9]


In [6]:
import random
"""
random_list = set()
while(len(set(random_list)) < 4):
    random_list.add(random.randint(0, 7)) # Generating integers between 0 and 100
random_list
"""
random_list = [0, 1, 5, 6]

In [7]:
prompts_selecionados_treinamento = []
for i in random_list:
    prompts_selecionados_treinamento.append( prompts[i] )
print(prompts_selecionados_treinamento)

['a-cultura-do-cancelamento-nas-redes-sociais', 'a-importancia-das-vacinas-para-sociedade', 'o-suicidio-entre-os-jovens-como-resolver-esse-problema', 'o-uso-agrotoxicos-no-brasil']


In [8]:
def selecionar_redacoes(df, prompts_selecionados):
    empty_df_train = pd.DataFrame()
    df_test = df.copy()
    for p in prompts_selecionados:
        s = df[df['id prompt'] == p]
        empty_df_train = pd.concat([empty_df_train, s])
        df_test = df_test[df_test['id prompt'] != p]
    return empty_df_train, df_test
    
filtrado_df_A, test_df_A = selecionar_redacoes(dfA, prompts_selecionados_treinamento)
filtrado_df_B, test_df_B = selecionar_redacoes(dfB, prompts_selecionados_treinamento)
df_unidos = pd.concat([filtrado_df_A, filtrado_df_B])

In [9]:
filtrado_df_A.to_csv('filtrado_A_train.csv', index=False)
filtrado_df_B.to_csv('filtrado_B_train.csv', index=False)
df_unidos.to_csv("Training.csv", index=False)

In [10]:
dic_sintax = {'Sintaxe inexistente': 0, 'Sintaxe deficitária': 1, 'Sintaxe regular':2, 'Sintaxe boa': 3, 'Sintaxe excelente': 4}
dic_desvios = {'Muitos desvios':0, 'Alguns desvios':1, 'Poucos desvios':2, 'Max dois desvios':3}

def adicionar_coluna(df, new_column, old_column, dic):
    #print(df)
    df[new_column] = df[old_column].map(dic)
    
def arrumar_dataframe(df, colunas_novas, colunas_velhas, dics, save_name):
    for c_n, c_v, dic in zip(colunas_novas, colunas_velhas, dics):
        print(c_n, c_v, dic)
        #print(dataset)
        adicionar_coluna(df, c_n, c_v, dic)
    df.rename(columns={'redação': 'essay_text'}, inplace=True)
    df['grades'] = df.apply(lambda row: [row['nota']*40, row['nota.1']*40, row['nota.2']*40, row['nota.3']*40, row['nota.4']*40], axis=1)
    df.to_csv(save_name, index=False)

colunas_novas = ['syntax', 'mistakes']
colunas_velhas = ['Sintaxe', 'Desvios']
dics = [dic_sintax, dic_desvios]
dfs = [filtrado_df_A, test_df_A]
nomes = ['Training-C1-A.csv', 'Test-C1-A.csv']

for df, nome in zip(dfs, nomes):
    arrumar_dataframe(df, colunas_novas, colunas_velhas, dics, nome)

syntax Sintaxe {'Sintaxe inexistente': 0, 'Sintaxe deficitária': 1, 'Sintaxe regular': 2, 'Sintaxe boa': 3, 'Sintaxe excelente': 4}
mistakes Desvios {'Muitos desvios': 0, 'Alguns desvios': 1, 'Poucos desvios': 2, 'Max dois desvios': 3}
syntax Sintaxe {'Sintaxe inexistente': 0, 'Sintaxe deficitária': 1, 'Sintaxe regular': 2, 'Sintaxe boa': 3, 'Sintaxe excelente': 4}
mistakes Desvios {'Muitos desvios': 0, 'Alguns desvios': 1, 'Poucos desvios': 2, 'Max dois desvios': 3}
